# AutoML: Train "the best" Time-Series Forecasting model for Retail Dataset.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [1]:
import warnings
import logging

# Suppress OpenTelemetry warnings
warnings.filterwarnings("ignore", message="Overriding of current")
warnings.filterwarnings("ignore", message="Attempting to instrument")

# Suppress Azure SDK telemetry logging
logging.getLogger("azure.core.pipeline.policies.http_logging_policy").setLevel(logging.WARNING)
logging.getLogger("azure.identity").setLevel(logging.WARNING)
logging.getLogger("opentelemetry").setLevel(logging.ERROR)

In [2]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import automl
from azure.ai.ml import Input

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [3]:
credential = DefaultAzureCredential()
ml_client = None
try:
    subscription_id = "57123c17-af1a-4ec2-9494-a214fb148bf4"
    resource_group = "admin-rg"
    workspace = "ml-demo-wksp-wus-01"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
except Exception as ex:
    print("Ex:", ex)

Class DeploymentTemplateOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


### Show Azure ML Workspace information

In [4]:
workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

output = {}
output["Workspace"] = ml_client.workspace_name
output["Subscription ID"] = ml_client.subscription_id
output["Resource Group"] = workspace.resource_group
output["Location"] = workspace.location
output

{'Workspace': 'ml-demo-wksp-wus-01',
 'Subscription ID': '57123c17-af1a-4ec2-9494-a214fb148bf4',
 'Resource Group': 'admin-rg',
 'Location': 'westus'}

# 2. Data

We will use [Retail data analytics] (https://www.kaggle.com/datasets/manjeetsingh/retaildataset) for model training. 
The data is stored in a tabular format and includes sales by store and department on a daily basis. 

With Azure Machine Learning MLTables you can keep a single copy of data in your storage, easily access data during model training, share data and collaborate with other users. 
Below, we will upload the data by creating an MLTable to be used for training.

## 2.1 Load and Explore Datasets

Load the three retail datasets: stores, features, and sales data.


In [6]:
import pandas as pd

# Load datasets
stores_df = pd.read_csv('../dataset/stores data-set.csv')
features_df = pd.read_csv('../dataset/Features data set.csv')
sales_df = pd.read_csv('../dataset/sales data-set.csv')

# Quick exploration
print(f"Stores: {stores_df.shape}")
print(f"Features: {features_df.shape}")
print(f"Sales: {sales_df.shape}")

print("\n--- Stores Data ---")
display(stores_df.head())

print("\n--- Features Data ---")
display(features_df.head())

print("\n--- Sales Data ---")
display(sales_df.head())


Stores: (45, 3)
Features: (8190, 12)
Sales: (421570, 5)

--- Stores Data ---


,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875



--- Features Data ---


,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False



--- Sales Data ---


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,05/02/2010,24924.50,False
1,1,1,12/02/2010,46039.49,True
2,1,1,19/02/2010,41595.55,False
3,1,1,26/02/2010,19403.54,False
4,1,1,05/03/2010,21827.90,False


## 2.2 Merge Datasets

Merge sales with stores (on Store) and then with features (on Store and Date).


In [7]:
# Merge sales with stores (on Store)
merged_df = sales_df.merge(stores_df, on='Store', how='left')

# Merge with features (on Store and Date)
merged_df = merged_df.merge(features_df, on=['Store', 'Date'], how='left', suffixes=('', '_feat'))

# Drop duplicate IsHoliday column from features
merged_df = merged_df.drop(columns=['IsHoliday_feat'])

print(f"Merged dataset shape: {merged_df.shape}")
print(f"\nColumns: {merged_df.columns.tolist()}")
display(merged_df.head())


Merged dataset shape: (421570, 16)

Columns: ['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Type', 'Size', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']


,Store,Dept,Date,Weekly_Sales,IsHoliday,Type,Size,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,1,05/02/2010,24924.50,False,A,151315,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,1,12/02/2010,46039.49,True,A,151315,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106
2,1,1,19/02/2010,41595.55,False,A,151315,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106
3,1,1,26/02/2010,19403.54,False,A,151315,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106
4,1,1,05/03/2010,21827.90,False,A,151315,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106


## 2.3 Feature Engineering

Create new features from date, handle missing MarkDown values, and encode categorical variables.


In [8]:
# Convert Date to datetime
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

# Extract date features
merged_df['Year'] = merged_df['Date'].dt.year
merged_df['Month'] = merged_df['Date'].dt.month
merged_df['Week'] = merged_df['Date'].dt.isocalendar().week
merged_df['DayOfWeek'] = merged_df['Date'].dt.dayofweek

# Handle missing MarkDown values (only available after Nov 2011)
markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
merged_df[markdown_cols] = merged_df[markdown_cols].fillna(0)

# Encode categorical: Store Type (A, B, C)
merged_df = pd.get_dummies(merged_df, columns=['Type'], prefix='StoreType')

# Create time series identifier
merged_df['ts_id'] = merged_df['Store'].astype(str) + '_' + merged_df['Dept'].astype(str)

print(f"Feature engineered dataset shape: {merged_df.shape}")
print(f"\nNew columns added:")
print(f"  - Date features: Year, Month, Week, DayOfWeek")
print(f"  - Store type encoded: StoreType_A, StoreType_B, StoreType_C")
print(f"  - Time series ID: ts_id")
print(f"\nMissing values in MarkDown columns (after fill):")
print(merged_df[markdown_cols].isnull().sum())
display(merged_df.head())


ValueError: time data "19/02/2010" doesn't match format "%m/%d/%Y", at position 2. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

## 2.4 Time-Based Train/Validation Split

Split data chronologically: train on data before 2012, validate on 2012 data.


In [ ]:
# Sort by date
merged_df = merged_df.sort_values(['Store', 'Dept', 'Date'])

# Time-based split: train on data before 2012, validate on 2012
train_df = merged_df[merged_df['Year'] < 2012].copy()
validation_df = merged_df[merged_df['Year'] >= 2012].copy()

print(f"Training set: {train_df.shape}")
print(f"Validation set: {validation_df.shape}")
print(f"\nTrain date range: {train_df['Date'].min()} to {train_df['Date'].max()}")
print(f"Validation date range: {validation_df['Date'].min()} to {validation_df['Date'].max()}")
print(f"\nTrain/Validation split ratio: {len(train_df)/(len(train_df)+len(validation_df))*100:.1f}% / {len(validation_df)/(len(train_df)+len(validation_df))*100:.1f}%")


## 2.5 Prepare Data for Azure ML AutoML

Rename columns to match AutoML expectations and save as MLTable format.


In [ ]:
import os

# Rename columns for AutoML compatibility
train_df = train_df.rename(columns={'Weekly_Sales': 'demand', 'Date': 'timeStamp'})
validation_df = validation_df.rename(columns={'Weekly_Sales': 'demand', 'Date': 'timeStamp'})

# Create output directories
os.makedirs('./data/training-mltable-folder', exist_ok=True)
os.makedirs('./data/validation-mltable-folder', exist_ok=True)

# Save as CSV (MLTable will reference these)
train_df.to_csv('./data/training-mltable-folder/train.csv', index=False)
validation_df.to_csv('./data/validation-mltable-folder/validation.csv', index=False)

print(f"Training data saved to: ./data/training-mltable-folder/train.csv")
print(f"Validation data saved to: ./data/validation-mltable-folder/validation.csv")
print(f"\nColumns in final datasets:")
print(train_df.columns.tolist())


In [ ]:
# Create MLTable YAML files for Azure ML

mltable_train = """paths:
  - file: ./train.csv
transformations:
  - read_delimited:
      delimiter: ','
      header: all_files_same_headers
"""

mltable_val = """paths:
  - file: ./validation.csv
transformations:
  - read_delimited:
      delimiter: ','
      header: all_files_same_headers
"""

with open('./data/training-mltable-folder/MLTable', 'w') as f:
    f.write(mltable_train)
    
with open('./data/validation-mltable-folder/MLTable', 'w') as f:
    f.write(mltable_val)

print("MLTable files created:")
print("  - ./data/training-mltable-folder/MLTable")
print("  - ./data/validation-mltable-folder/MLTable")


In [ ]:
# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="./data/training-mltable-folder"
)

# Training MLTable defined locally, with local data to be uploaded
my_validation_data_input = Input(
    type=AssetTypes.MLTABLE, path="./data/validation-mltable-folder"
)

# WITH REMOTE PATH
# my_training_data_input  = Input(type=AssetTypes.MLTABLE, path="azureml://datastores/workspaceblobstore/paths/my-forecasting-mltable")

For documentation on creating your own MLTable assets for jobs beyond this notebook:
- https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-mltable details how to write MLTable YAMLs (required for each MLTable asset).
- https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-data-assets?tabs=Python-SDK covers how to work with them in the v2 CLI/SDK.

# 3. Configure and run the AutoML Forecasting training job
In this section we will configure and run the AutoML job, for training the model.

## 3.1 Configure the job through the forecasting() factory function

### forecasting() function parameters:

The `forecasting()` factory function allows user to configure AutoML for the forecasting task for the most common scenarios with the following properties.

- `target_column_name` - The name of the column to target for predictions. It must always be specified. This parameter is applicable to 'training_data', 'validation_data' and 'test_data'.
- `primary_metric` - The metric that AutoML will optimize for model selection.
- `training_data` - The data to be used for training. It should contain both training feature columns and a target column. Optionally, this data can be split for segregating a validation or test dataset. 
You can use a registered MLTable in the workspace using the format '<mltable_name>:<version>' OR you can use a local file or folder as a MLTable. For e.g Input(mltable='my_mltable:1') OR Input(mltable=MLTable(local_path="./data"))
The parameter 'training_data' must always be provided.
- `compute` - The compute on which the AutoML job will run. In this example we are using serverless compute. You can alternatively use a compute cluster in the workspace. 
- `name` - The name of the Job/Run. This is an optional property. If not specified, a random name will be generated.
- `experiment_name` - The name of the Experiment. An Experiment is like a folder with multiple runs in Azure ML Workspace that should be related to the same logical machine learning experiment.

### set_limits() parameters:
This is an optional configuration method to configure limits parameters such as timeouts.     
    
- timeout_minutes - Maximum amount of time in minutes that the whole AutoML job can take before the job terminates. This timeout includes setup, featurization and training runs but does not include the ensembling and model explainability runs at the end of the process since those actions need to happen once all the trials (children jobs) are done. If not specified, the default job's total timeout is 6 days (8,640 minutes). To specify a timeout less than or equal to 1 hour (60 minutes), make sure your dataset's size is not greater than 10,000,000 (rows times column) or an error results.

- trial_timeout_minutes - Maximum time in minutes that each trial (child job) can run for before it terminates. If not specified, a value of 1 month or 43200 minutes is used.
    
- max_trials - The maximum number of trials/runs each with a different combination of algorithm and hyperparameters to try during an AutoML job. If not specified, the default is 1000 trials. If using 'enable_early_termination' the number of trials used can be smaller.
    
- max_concurrent_trials - Represents the maximum number of trials (children jobs) that would be executed in parallel. It's a good practice to match this number with the number of nodes your cluster.
    
- enable_early_termination - Whether to enable early termination if the score is not improving in the short term.

## Specialized Forecasting Parameters
To define forecasting parameters for your experiment training, you can leverage the .set_forecast_settings() method. 
The table below details the forecasting parameters we will be passing into our experiment.

|Property|Description|
|-|-|
|**time_column_name**|The name of your time column.|
|**forecast_horizon**|The forecast horizon is how many periods forward you would like to forecast. This integer horizon is in units of the timeseries frequency (e.g. daily, weekly).|
|**frequency**|Forecast frequency. This optional parameter represents the period with which the forecast is desired, for example, daily, weekly, yearly, etc. Use this parameter for the correction of time series containing irregular data points or for padding of short time series. The frequency needs to be a pandas offset alias. Please refer to [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) for more information.

# Advanced Forecasting Training <a id="advanced_training"></a>
### Using lags and rolling window features
This training is also using the **target lags**, that is the previous values of the target variables, meaning the prediction uses a horizon. We therefore must still specify the `forecast_horizon` that the model will learn to forecast. The `target_lags` keyword specifies how far back we will construct the lags of the target variable, and the `target_rolling_window_size` specifies the size of the rolling window over which we will generate the `max`, `min` and `sum` features.

This notebook uses the .set_training(blocked_training_algorithms=...) parameter to exclude some models that take a longer time to train on this dataset.  You can choose to remove models from the blocked_training_algorithms list but you may need to increase the trial_timeout_minutes parameter value to get results.

In [ ]:
# general job parameters
max_trials = 5
exp_name = "dpv2-forecasting-experiment"

In [ ]:
# Create the AutoML forecasting job with the related factory-function.

forecasting_job = automl.forecasting(
    # name="dpv2-forecasting-job-02",
    experiment_name=exp_name,
    training_data=my_training_data_input,
    # validation_data = my_validation_data_input,
    target_column_name="demand",
    primary_metric="NormalizedRootMeanSquaredError",
    n_cross_validations="auto",
    enable_model_explainability=True,
    tags={"my_custom_tag": "My custom value"},
)

# Limits are all optional
forecasting_job.set_limits(
    timeout_minutes=600,
    trial_timeout_minutes=20,
    max_trials=max_trials,
    # max_concurrent_trials = 4,
    # max_cores_per_trial: -1,
    enable_early_termination=True,
)

# Specialized properties for Time Series Forecasting training
forecasting_job.set_forecast_settings(
    time_column_name="timeStamp",
    forecast_horizon=52,  # 52 weeks for yearly forecast
    frequency="W",        # Weekly frequency (matches our retail data)
    target_lags=[1, 2, 4],  # Lag features for 1, 2, and 4 weeks back
    target_rolling_window_size=4,
    time_series_id_column_names=["Store", "Dept"],  # Identify each store-department time series
    # short_series_handling_config=ShortSeriesHandlingConfiguration.DROP,
    # use_stl="season",
    # seasonality=3,
)

# Training properties are optional
forecasting_job.set_training(blocked_training_algorithms=["ExtremeRandomTrees"])

# Featurization properties are optional
# forecasting_job.set_featurization(# drop_columns=["not_needed_column"], # Optional
#                                   # enable_dnn_featurization=True         # Enable if there are text columns
#                                     )

## Run the Command
Using the `MLClient` created earlier, we will now run this Command in the workspace.

In [ ]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    forecasting_job
)  # submit the job to the backend

print(f"Created job: {returned_job}")

In [ ]:
ml_client.jobs.stream(returned_job.name)

# Next Steps
You can see further examples of other AutoML tasks such as Image-Classification, Image-Object-Detection, NLP-Text-Classification, Time-Series-Forcasting, etc.